In [113]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time

# --------------------------------------------------------------------------
def calculateSyndrome(codeword, parityCheck):
    # Izračunavanje sindroma parnosti
    syndrome = 0

    for bitPosition in parityCheck:
        syndrome ^= codeword[bitPosition]

    return syndrome



# ---------------------------------------------------------------------------

def calculateSyndromeGDBFM(codeword, parityCheck):
    # Izračunavanje sindroma parnosti
    syndrome = 1

    for bitPosition in parityCheck:
        syndrome *= codeword[bitPosition]

    return syndrome


# --------------------------------------------------------------------------
def findBitFlipPositions(codeword, syndrome, parityCheck):
    # Pronalaženje pozicija bitova koje treba invertovati
    flipPositions = []
    for bitPosition in parityCheck:
        if syndrome != codeword[bitPosition]:
            flipPositions.append(bitPosition)

    return flipPositions

# --------------------------------------------------------------------------

def findParityPositions(LDPCmatrix):
    return [list(np.nonzero(row)[0]) for row in LDPCmatrix]  # vraca se LDPC Code u daljnjem

# --------------------------------------------------------------------------

def recnik(LDPCCode, H):
    dict = {}
    values = []
    for i in range(len(H[0])):
        for parity in LDPCCode:
            if i in parity:
                values.append(LDPCCode.index(parity))
        dict.update({i:values})
        values = []
    return dict

# -------------------------------------------------------------------------

def decisionMaking(codeword, recnik, iterator, LDPCCode):
    syndromeList = []

    parity = recnik[iterator]
    for i in parity:
        syndromeList.append(calculateSyndrome(codeword, LDPCCode[i]))
    
    if syndromeList.count(1) > syndromeList.count(-1):
        return 1
    else:
        return -1


# --------------------------------------------------------------------------

def bitFlippingDecoder(codeword, maxIterations, recnik, LDPCCode):
    # Inicijalizacija
    decodedWord = codeword.copy()
    numIterations = 0

    while numIterations < maxIterations:
        converged = True

        # Iteracija kroz sve parnosti u kodu
        for i in range(len(decodedWord)):
            newSyndrome = decisionMaking(decodedWord, recnik, i, LDPCCode)

            # Provera i ispravka grešaka
            if newSyndrome == -1:
                decodedWord[i] *= -1
            
                converged = False
        numIterations += 1

        if converged: break

    return decodedWord


# -------------------------------------------------------------------------

def sign(x):
    return (x > 0) - (x < 0)

# -------------------------------------------------------------------------
# -------------------------------------------------------------------------
# -------------------------------------------------------------------------


In [120]:
matrix = np.array([
    [1, 0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [1, 0, 0, 0, 0, 1]
])

parity_positions = findParityPositions(matrix)  # 'parity_positions' stores the positions of ones in the parity-check matrix.
codeword = [1, 1, 0, 0, 1, 0]
codeword_copy = codeword.copy()

for i in range(len(codeword)):
    if codeword[i] == 0:
        codeword[i] = 1  # Convert 0 to 1.
    else:
        codeword[i] = -1  # Convert 1 to -1.

print(codeword)

parity_dictionary = recnik(parity_positions, matrix)

num_tests = 1

bit_flipping_errors = []

for k in range(num_tests):
    codeword_copy = codeword.copy()
    print(codeword_copy)
    decoded_sequence_BF = bitFlippingDecoder(codeword_copy, 4, parity_dictionary, parity_positions)

    print("decoded_sequence_BF", decoded_sequence_BF)
    if decoded_sequence_BF.count(-1) == 0:
        bit_flipping_errors.append(0)
    else:
        bit_flipping_errors.append(1)

[-1, -1, 1, 1, -1, 1]
[-1, -1, 1, 1, -1, 1]
decoded_sequence_BF [-1, -1, 1, 1, -1, 1]


In [112]:
parity_dictionary

{0: [0, 2], 1: [1], 2: [], 3: [0, 1], 4: [], 5: [2]}

In [105]:
parity_dictionary

{0: [0, 2], 1: [1], 2: [], 3: [0, 1], 4: [], 5: [2]}

In [104]:
decoded_sequence_BF

[-1, -1, 1, -1, -1, 1]

In [89]:
codeword = [1, 0, 0, 1, 0, 0]
matrix @ codeword

array([2, 1, 1])

In [ ]:
matrix = np.array([
    [1, 0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [1, 0, 0, 0, 0, 1]
])

parity_positions = findParityPositions(matrix)  # 'parity_positions' stores the positions of ones in the parity-check matrix.
codeword = [1, 0, 0, 1, 0, 0]

In [20]:
FER_bit_flipping

[0.01455]

In [43]:
import numpy as np
import ldpc.codes
from ldpc import BpDecoder

H=np.array([
    [1, 0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [1, 0, 0, 0, 0, 1]
])
n=H.shape[1] #the codeword length

bpd = BpDecoder(
    H, #the parity check matrix
    error_rate=0.1, # the error rate on each bit
    max_iter=n, #the maximum iteration depth for BP
    bp_method="product_sum", #BP method. The other option is `minimum_sum'
)

In [44]:
H

array([[1, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1]])

In [47]:
received_vector=np.array([1, 0, 0, 0, 1, 0])
decoded_codeword=bpd.decode(received_vector)

print(decoded_codeword)


[0 0 0 0 1 0]


In [48]:
H @ decoded_codeword

array([0, 0, 0])